In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import numpy as np


In [2]:
# Import API key
from config2 import g_key

In [3]:
# Configure gmaps API key
gmaps.configure(api_key=g_key)

# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv(r"C:\Users\ajibo\World_Weather_Analysis\Weather Database\WeatherPy_Database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Jamestown,US,42.0970,-79.2353,42.39,51,1,8.99,clear sky
1,1,Kayasula,RU,44.3250,45.0000,38.62,77,96,14.23,overcast clouds
2,2,North Bend,US,43.4065,-124.2243,49.86,98,99,16.24,overcast clouds
3,3,Mount Gambier,AU,-37.8333,140.7667,69.71,64,31,13.80,scattered clouds
4,4,Arlit,NE,18.7369,7.3853,69.58,8,87,7.18,overcast clouds


In [4]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))


What is the minimum temperature you would like for your trip? 78
What is the maximum temperature you would like for your trip? 90


In [5]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method
filtered_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
filtered_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
9,9,Kimbe,PG,-5.5502,150.1429,78.22,92,100,1.30,overcast clouds
17,17,Oranjestad,AW,12.5240,-70.0270,82.35,83,40,3.00,scattered clouds
22,22,Vaini,TO,-21.2000,-175.2000,82.56,83,40,13.80,light rain
23,23,Tonj,SS,7.2737,28.6822,79.18,29,61,3.42,broken clouds
31,31,Georgetown,MY,5.4112,100.3354,79.90,82,20,0.00,few clouds
39,39,Noumea,NC,-22.2763,166.4572,84.18,80,100,2.48,light rain
40,40,Broken Hill,AU,-31.9500,141.4333,83.91,15,5,12.66,clear sky
63,63,Catamarca,AR,-28.4696,-65.7852,83.59,55,29,8.01,scattered clouds
64,64,Hilo,US,19.7297,-155.0900,79.11,58,75,5.75,broken clouds
67,67,Moron,AR,-34.6534,-58.6198,78.69,72,84,5.44,broken clouds


In [6]:
# 4a. Determine if there are any empty rows.
filtered_cities_df.count()

City_ID                131
City                   131
Country                131
Lat                    131
Lng                    131
Max Temp               131
Humidity               131
Cloudiness             131
Wind Speed             131
Current Description    131
dtype: int64

In [7]:
filtered_cities_df.isnull().sum()

City_ID                0
City                   0
Country                0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [8]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = filtered_cities_df
clean_df

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
9,9,Kimbe,PG,-5.5502,150.1429,78.22,92,100,1.30,overcast clouds
17,17,Oranjestad,AW,12.5240,-70.0270,82.35,83,40,3.00,scattered clouds
22,22,Vaini,TO,-21.2000,-175.2000,82.56,83,40,13.80,light rain
23,23,Tonj,SS,7.2737,28.6822,79.18,29,61,3.42,broken clouds
31,31,Georgetown,MY,5.4112,100.3354,79.90,82,20,0.00,few clouds
...,...,...,...,...,...,...,...,...,...,...
666,666,Paracuru,BR,-3.4100,-39.0306,82.42,77,68,12.55,broken clouds
672,672,Nicoya,CR,10.1483,-85.4520,79.72,71,28,6.80,scattered clouds
675,675,Chirongui,YT,-12.9319,45.1483,80.53,72,76,8.50,broken clouds
690,690,Isangel,VU,-19.5500,169.2667,83.21,77,100,10.02,overcast clouds


In [9]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = filtered_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()
# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)


,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
9,Kimbe,PG,78.22,overcast clouds,-5.5502,150.1429,
17,Oranjestad,AW,82.35,scattered clouds,12.5240,-70.0270,
22,Vaini,TO,82.56,light rain,-21.2000,-175.2000,
23,Tonj,SS,79.18,broken clouds,7.2737,28.6822,
31,Georgetown,MY,79.90,few clouds,5.4112,100.3354,
39,Noumea,NC,84.18,light rain,-22.2763,166.4572,
40,Broken Hill,AU,83.91,clear sky,-31.9500,141.4333,
63,Catamarca,AR,83.59,scattered clouds,-28.4696,-65.7852,
64,Hilo,US,79.11,broken clouds,19.7297,-155.0900,
67,Moron,AR,78.69,broken clouds,-34.6534,-58.6198,


In [10]:
# 6a. Set parameters to search for hotels  within 5000 meters.
params = {
        "radius": 5000,
        "type": "lodging",
        "key": g_key,
       
}



# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    lat = row['Lat']
    lng = row['Lng']

    # 6c. Get latitude and longitude from DataFrame
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
  
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
        
    except (IndexError): 
        print("No hotel found...skipping.")
        
        
hotel_df.head(10)
        

No hotel found...skipping.
No hotel found...skipping.
No hotel found...skipping.
No hotel found...skipping.
No hotel found...skipping.
No hotel found...skipping.
No hotel found...skipping.
No hotel found...skipping.
No hotel found...skipping.
No hotel found...skipping.
No hotel found...skipping.
No hotel found...skipping.
No hotel found...skipping.
No hotel found...skipping.
No hotel found...skipping.
No hotel found...skipping.
No hotel found...skipping.
No hotel found...skipping.
No hotel found...skipping.
No hotel found...skipping.
No hotel found...skipping.
No hotel found...skipping.
No hotel found...skipping.
No hotel found...skipping.
No hotel found...skipping.
No hotel found...skipping.
No hotel found...skipping.
No hotel found...skipping.
No hotel found...skipping.
No hotel found...skipping.
No hotel found...skipping.
No hotel found...skipping.
No hotel found...skipping.
No hotel found...skipping.
No hotel found...skipping.
No hotel found...skipping.
No hotel found...skipping.
N

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
9,Kimbe,PG,78.22,overcast clouds,-5.5502,150.1429,
17,Oranjestad,AW,82.35,scattered clouds,12.5240,-70.0270,
22,Vaini,TO,82.56,light rain,-21.2000,-175.2000,
23,Tonj,SS,79.18,broken clouds,7.2737,28.6822,
31,Georgetown,MY,79.90,few clouds,5.4112,100.3354,
39,Noumea,NC,84.18,light rain,-22.2763,166.4572,
40,Broken Hill,AU,83.91,clear sky,-31.9500,141.4333,
63,Catamarca,AR,83.59,scattered clouds,-28.4696,-65.7852,
64,Hilo,US,79.11,broken clouds,19.7297,-155.0900,
67,Moron,AR,78.69,broken clouds,-34.6534,-58.6198,


In [11]:
  # 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = (hotel_df.replace(r'^\s*$', np.nan, regex=True)).dropna()
clean_hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
